In [1]:
from dotenv import load_dotenv
load_dotenv("/home/marshath/play/chainlink/algovate/.env")

In [121]:
import time
import pickle
import pandas as pd

from langchain.vectorstores import FAISS
from langchain.text_splitter import TokenTextSplitter, RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import FlareChain, RetrievalQAWithSourcesChain
from langchain.chains.router import MultiRetrievalQAChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from tqdm.notebook import tqdm

from algovate.evaluate import run_eval

In [78]:
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [52]:
with open("/home/marshath/play/chainlink/algovate/algovate/data/documents.pkl", "rb") as f:
    documents = pickle.load(f)


splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=50)
documents = splitter.split_documents(documents)

In [53]:
vectorstore = FAISS.from_documents(documents, embedding=OpenAIEmbeddings())

### Flare

In [54]:
flare = FlareChain.from_llm(
    ChatOpenAI(temperature=0), 
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    max_generation_len=164,
    min_prob=.3,
)

In [30]:
flare.run("What is the proof of reserve address for CacheGold PoR USD on Goerli Testnet?")

' The proof of reserve address for CacheGold PoR USD on Goerli Testnet is 0xDe9C980F79b636B46b9c3bc04cfCC94A29D18D19.\n\n'

### MultiRouter

In [59]:
with open("/home/marshath/play/chainlink/algovate/algovate/data/documents.pkl", "rb") as f:
    documents = pickle.load(f)

In [69]:
def name_desc(
    documents):
    splitter = TokenTextSplitter(chunk_size=250, chunk_overlap=50)
    documents = splitter.split_documents(documents)
    vectorstore = FAISS.from_documents(documents, embedding=OpenAIEmbeddings())
    retriever = vectorstore.as_retriever()

    template = """
    Given the following extracted parts of a long document and a question, create a name for the document in less than 4 words and description in 20 words.

    QUESTION: {question}
    =========
    {summaries}
    =========
    Name:
    Description:
    """

    PROMPT = PromptTemplate(template=template, 
                            input_variables=["summaries", "question"])
    
    chain = RetrievalQAWithSourcesChain.from_chain_type(ChatOpenAI(temperature=0.), 
                                                        chain_type="stuff", 
                                                        retriever=retriever,
                                                        chain_type_kwargs={"prompt":PROMPT})
    result = chain({
    "question": "What is this document about",
    }, return_only_outputs=True)

    try:
        name = result["answer"].split(":")[1].split("\n")[0].strip()
        desc = result["answer"].split(":")[2].split("\n")[0].strip()
    except:
        name = ""
        desc = ""
    return name, desc, retriever

In [1]:
# for pair, doc in tqdm(zip(retriever_infos, documents), total=len(documents)):
#     splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
#     doc_chunked = splitter.split_documents([doc])
#     vectorstore = FAISS.from_documents(doc_chunked, embedding=OpenAIEmbeddings())
#     retriever = vectorstore.as_retriever()
#     pair["retriever"] = retriever

In [70]:
retriever_infos = []
for doc in tqdm(documents, total=len(documents)):
    name, desc, retriever = name_desc([doc])
    retriever_infos.append({"name":name, "description":desc, "retriever":retriever})

  0%|          | 0/151 [00:00<?, ?it/s]

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID b60592f13f023462e9beb553e2c3cdff in your message.).
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 1d7a59dbc95c7067ab0c9ced0644fd9c in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please incl

In [123]:
with open("/home/marshath/play/chainlink/algovate/algovate/data/retriever_infos.pkl", "wb") as f:
    pickle.dump(retriever_infos, f)

In [124]:
chain = MultiRetrievalQAChain.from_retrievers(ChatOpenAI(), retriever_infos)

In [125]:
chain.run("What is chainlink?")

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "user", "content": "Given a query to a question answering system select the system best suited for the input. You will be given the names of the available systems and a description of what questions the system is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response.\\n\\n<< FORMATTING >>\\nReturn a markdown code snippet with a JSON object formatted to look like:\\n```json\\n{\\n    \\"destination\\": string \\\\ name of the question answering system to use or \\"DEFAULT\\"\\n    \\"next_inputs\\": string \\\\ a potentially modified version of the original input\\n}\\n```\\n\\nREMEMBER: \\"destination\\" MUST be one of the candidate prompt names specified below OR it can be \\"DEFAULT\\" if the input is not well suited for any of the candidate prompts.

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 5095 tokens. Please reduce the length of the messages.

In [71]:
ground_truth_path = "/home/marshath/play/chainlink/algovate/algovate/data/ground_truths.pkl"
with open(ground_truth_path, "rb") as f:
    ground_truths = pickle.load(f)

In [99]:
result_df = pd.DataFrame()
for i in range(len(ground_truths)):
    ground_truth = ground_truths[12]
    try:
        predictions = []
        retrieved_docs = []
        gt_dataset = []
        latency = []

        start_time = time.time()
        prediction = chain.run(ground_truth["question"])
        break
    except Exception as e:
        print(e)
        pass
        # predictions.append(chain(ground_truth))
        # gt_dataset.append(ground_truth)
        # end_time = time.time()
        # elapsed_time = end_time - start_time
        # latency.append(elapsed_time)
        # d = pd.DataFrame(predictions)
        # try:
        #     graded_answers = grade_model_answer(
        #     gt_dataset, predictions, grading_type=grading_type)
        #     d['answerScore'] = [g['text'] for g in graded_answers]
        #     d['answerScore'] = [{'score': 1 if "Incorrect" not in text else 0,
        #                      'justification': text} for text in d['answerScore']]
        # except:
        #     graded_answers = []
        #     d['answerScore'] = [{"score":"N/A", "justification":"N/A"}]
        # try:
        #     graded_retrieval = grade_model_retrieval(
        #     gt_dataset, retrieved_docs, grading_type=grading_type)
        #     d['retrievalScore'] = [g['text'] for g in graded_retrieval]
        #     d['retrievalScore'] = [{'score': 1 if "Incorrect" not in text else 0,
        #                         'justification': text} for text in d['retrievalScore']]
        # except:
        #     graded_retrieval = []
        #     d['retrievalScore'] = [{"score":"N/A", "justification":"N/A"}]
        
        # d['latency'] = latency
        # d_dict = d.to_dict('records')

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "user", "content": "Given a query to a question answering system select the system best suited for the input. You will be given the names of the available systems and a description of what questions the system is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response.\\n\\n<< FORMATTING >>\\nReturn a markdown code snippet with a JSON object formatted to look like:\\n```json\\n{\\n    \\"destination\\": string \\\\ name of the question answering system to use or \\"DEFAULT\\"\\n    \\"next_inputs\\": string \\\\ a potentially modified version of the original input\\n}\\n```\\n\\nREMEMBER: \\"destination\\" MUST be one of the candidate prompt names specified below OR it can be \\"DEFAULT\\" if the input is not well suited for any of the candidate prompts.

This model's maximum context length is 4097 tokens. However, your messages resulted in 5095 tokens. Please reduce the length of the messages.


DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 400 281
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=11 request_id=1bd81a9adaa73d3a50ec75bdc8048bd1 response_code=400
INFO:openai:error_code=context_length_exceeded error_message="This model's maximum context length is 4097 tokens. However, your messages resulted in 5095 tokens. Please reduce the length of the messages." error_param=messages error_type=invalid_request_error message='OpenAI API error received' stream_error=False
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "user", "content": "Given a query to a question answering system select the system best suited for the input. You will be given the names of the available systems and a description of what questions the system is best suited for. You may also revise t

This model's maximum context length is 4097 tokens. However, your messages resulted in 5095 tokens. Please reduce the length of the messages.


DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 400 281
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=11 request_id=1aaa34c218f9718937bbebffd154790e response_code=400
INFO:openai:error_code=context_length_exceeded error_message="This model's maximum context length is 4097 tokens. However, your messages resulted in 5095 tokens. Please reduce the length of the messages." error_param=messages error_type=invalid_request_error message='OpenAI API error received' stream_error=False
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "user", "content": "Given a query to a question answering system select the system best suited for the input. You will be given the names of the available systems and a description of what questions the system is best suited for. You may also revise t

...

This model's maximum context length is 4097 tokens. However, your messages resulted in 5095 tokens. Please reduce the length of the messages.


DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 429 354
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=None request_id=5a5aa6c0af309d980b49a1e749fdde8f response_code=429
INFO:openai:error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-apfNELnY4pAbHrx6LItJCss8 on tokens per min. Limit: 90000 / min. Current: 88516 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "user", "content": "Given a query to a question answering system select the system best suited for the input. You will be given the names of the available systems and a description of what que

This model's maximum context length is 4097 tokens. However, your messages resulted in 5095 tokens. Please reduce the length of the messages.


DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 429 354
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=None request_id=cd664cf15ae3de5e041f5139e11ee665 response_code=429
INFO:openai:error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-apfNELnY4pAbHrx6LItJCss8 on tokens per min. Limit: 90000 / min. Current: 89000 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "user", "content": "Given a query to a question answering system select the system best suited for the input. You will be given the names of the available systems and a description of what que

This model's maximum context length is 4097 tokens. However, your messages resulted in 5095 tokens. Please reduce the length of the messages.


In [97]:
prediction

NameError: name 'prediction' is not defined